In [2]:
import sqlite3
import pandas as pd
from datetime import datetime

def query_database(db_path='option_strategies.db'):
    """
    Query the database and display information about the stored data
    
    Parameters:
    db_path (str): Path to the SQLite database file
    """
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        
        # Create a cursor and pandas connection
        cursor = conn.cursor()
        
        # Check if the database has the required table
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='option_strategies'")
        if not cursor.fetchone():
            print(f"Error: Database {db_path} does not contain the option_strategies table.")
            conn.close()
            return
        
        # Get record count
        cursor.execute("SELECT COUNT(*) FROM option_strategies")
        total_records = cursor.fetchone()[0]
        
        print(f"Database: {db_path}")
        print(f"Total records: {total_records}")
        
        if total_records == 0:
            print("No records found in the database.")
            conn.close()
            return
        
        # Get count by strategy type
        print("\nRecords by strategy type:")
        cursor.execute("SELECT strategy_type, COUNT(*) as count FROM option_strategies GROUP BY strategy_type")
        results = cursor.fetchall()
        for strategy, count in results:
            print(f"  {strategy}: {count} records")
        
        # Get count by tab
        print("\nRecords by strategy and tab:")
        cursor.execute("SELECT strategy_type, tab_name, COUNT(*) as count FROM option_strategies GROUP BY strategy_type, tab_name")
        results = cursor.fetchall()
        for strategy, tab, count in results:
            print(f"  {strategy} - {tab}: {count} records")
        
        # Get most recent scrape date
        cursor.execute("SELECT MAX(scrape_date) FROM option_strategies")
        most_recent = cursor.fetchone()[0]
        print(f"\nMost recent scrape: {most_recent}")
        
        # Get ticker distribution
        print("\nTop 10 tickers:")
        cursor.execute("""
            SELECT ticker, COUNT(*) as count 
            FROM option_strategies 
            WHERE ticker != 'N/A' 
            GROUP BY ticker 
            ORDER BY count DESC 
            LIMIT 10
        """)
        results = cursor.fetchall()
        for ticker, count in results:
            print(f"  {ticker}: {count} records")
        
        # Display sample records
        print("\nSample records (most recent 5):")
        df = pd.read_sql_query("""
            SELECT strategy_type, tab_name, ticker, trigger_price, strike_price, estimated_premium, options_expiry_date
            FROM option_strategies
            ORDER BY scrape_date DESC
            LIMIT 5
        """, conn)
        
        print(df)
        
        conn.close()
        
    except Exception as e:
        print(f"Error querying database: {str(e)}")

def export_to_csv(db_path='option_strategies.db', output_file=None):
    """
    Export the database to a CSV file
    
    Parameters:
    db_path (str): Path to the SQLite database file
    output_file (str): Path for the output CSV file. If None, a timestamped filename is used.
    
    Returns:
    str: Path to the created CSV file
    """
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        
        # Create a cursor
        cursor = conn.cursor()
        
        # Check if the database has the required table
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='option_strategies'")
        if not cursor.fetchone():
            print(f"Error: Database {db_path} does not contain the option_strategies table.")
            conn.close()
            return None
        
        # Create output filename if not provided
        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"option_strategies_export_{timestamp}.csv"
        
        # Query all data
        df = pd.read_sql_query("SELECT * FROM option_strategies", conn)
        
        # Export to CSV
        df.to_csv(output_file, index=False)
        
        print(f"Successfully exported {len(df)} records to {output_file}")
        
        conn.close()
        return output_file
        
    except Exception as e:
        print(f"Error exporting database: {str(e)}")
        return None

if __name__ == "__main__":
    # Query the database
    query_database()
    
    # Uncomment to export to CSV
    # export_to_csv()

Database: option_strategies.db
Total records: 18

Records by strategy type:
  Bear Call: 5 records
  Bull Put: 13 records

Records by strategy and tab:
  Bear Call - Mild Risk 95-97% accuracy > longer expiry: 1 records
  Bear Call - Mild Risk 95-97% accuracy > shorter expiry: 2 records
  Bear Call - Minimal Risk 97-99% accuracy > longer expiry: 1 records
  Bear Call - Minimal Risk 97-99% accuracy > shorter expiry: 1 records
  Bull Put - Mild Risk 95-97% accuracy > longer expiry: 4 records
  Bull Put - Mild Risk 95-97% accuracy > shorter expiry: 4 records
  Bull Put - Minimal Risk 97-99% accuracy > longer expiry: 3 records
  Bull Put - Minimal Risk 97-99% accuracy > shorter expiry: 2 records

Most recent scrape: 2025-03-08T06:06:54.242743

Top 10 tickers:
  UNH: 3 records
  AMZN: 2 records
  NVDA: 2 records
  QQQ: 2 records
  TQQQ: 2 records
  WMT: 2 records
  GOOGL: 1 records
  MA: 1 records
  META: 1 records
  MSFT: 1 records

Sample records (most recent 5):
  strategy_type           

In [3]:
df

NameError: name 'df' is not defined

In [5]:
import sqlite3
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Connect to the SQLite database
def get_strategies_by_date(scrape_date=None, db_path='option_strategies.db'):
    """
    Query option strategies from the database for a specific scrape date
    
    Parameters:
    scrape_date (str): Specific scrape date to query for (ISO format, e.g. '2025-03-08T14:30:45.123456')
                       If None, returns most recent scrape date's data
    db_path (str): Path to the SQLite database file
    
    Returns:
    pandas.DataFrame: DataFrame containing the matched records
    """
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        
        # If no specific date provided, get the most recent date
        if scrape_date is None:
            query = "SELECT MAX(scrape_date) FROM option_strategies"
            most_recent = pd.read_sql_query(query, conn).iloc[0, 0]
            scrape_date = most_recent
            print(f"Using most recent scrape date: {scrape_date}")
        
        # Query using date prefix (to handle time portion variations)
        date_prefix = scrape_date.split('T')[0] if 'T' in scrape_date else scrape_date
        query = f"""
            SELECT * FROM option_strategies 
            WHERE scrape_date LIKE '{date_prefix}%'
            ORDER BY strategy_type, tab_name
        """
        
        # Execute query and load into DataFrame
        df = pd.read_sql_query(query, conn)
        
        # Close connection
        conn.close()
        
        # Display summary information
        if len(df) > 0:
            print(f"Found {len(df)} records from {date_prefix}")
            print(f"Strategies: {', '.join(df['strategy_type'].unique())}")
        else:
            print(f"No records found for date {date_prefix}")
        
        return df
        
    except Exception as e:
        print(f"Error querying database: {str(e)}")
        return pd.DataFrame()

# Example usage:
# 1. Get strategies from most recent scrape
#  recent_strategies = get_strategies_by_date()

# 2. Get strategies from a specific date
# specific_date = "2025-03-08"
# specific_strategies = get_strategies_by_date(specific_date)

# Display sample of the data
# if not recent_strategies.empty:
#     display(recent_strategies.head())
    
#     # Create a simple visualization of strategy distribution
#     strategy_counts = recent_strategies['strategy_type'].value_counts()
#     plt.figure(figsize=(10, 6))
#     strategy_counts.plot(kind='bar')
#     plt.title('Distribution of Option Strategies')
#     plt.xlabel('Strategy Type')
#     plt.ylabel('Number of Records')
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()
    
#     # Show ticker distribution (top 10)
#     if 'ticker' in recent_strategies.columns:
#         ticker_counts = recent_strategies['ticker'].value_counts().head(10)
#         plt.figure(figsize=(10, 6))
#         ticker_counts.plot(kind='bar')
#         plt.title('Top 10 Tickers in Option Strategies')
#         plt.xlabel('Ticker')
#         plt.ylabel('Number of Records')
#         plt.xticks(rotation=45)
#         plt.tight_layout()
#         plt.show()


specific_date = "2025-03-08"
specific_strategies = get_strategies_by_date(specific_date)
specific_strategies

Found 18 records from 2025-03-08
Strategies: Bear Call, Bull Put


,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,estimated_premium,item_id,options_expiry_date,date_info
0,4,2025-03-08T06:06:29.450187,Bear Call,Mild Risk 95-97% accuracy > longer expiry,UNH,504.95,sell 540.0 - buy 550.0,79,1,2025-03-28,March
1,1,2025-03-08T06:06:22.557833,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,GOOGL,182.37,sell 192.5 - buy 202.5,76,1,2025-03-21,March
2,2,2025-03-08T06:06:22.626663,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,MSFT,404.89,sell 425.0 - buy 435.0,124,2,2025-03-21,March
3,5,2025-03-08T06:06:32.937225,Bear Call,Minimal Risk 97-99% accuracy > longer expiry,UNH,507.27,sell 545.0 - buy 555.0,70,1,2025-03-28,March
4,3,2025-03-08T06:06:26.040499,Bear Call,Minimal Risk 97-99% accuracy > shorter expiry,UNH,507.59,sell 535.0 - buy 545.0,93,1,2025-03-21,March
5,12,2025-03-08T06:06:50.054943,Bull Put,Mild Risk 95-97% accuracy > longer expiry,AMZN,193.36,sell 182.5 - buy 172.5,180,1,2025-03-28,March
6,13,2025-03-08T06:06:50.198675,Bull Put,Mild Risk 95-97% accuracy > longer expiry,META,589.67,sell 540.0 - buy 530.0,97,2,2025-03-28,March
7,14,2025-03-08T06:06:50.312069,Bull Put,Mild Risk 95-97% accuracy > longer expiry,NVDA,104.72,sell 95.0 - buy 85.0,158,3,2025-03-28,March
8,15,2025-03-08T06:06:50.456818,Bull Put,Mild Risk 95-97% accuracy > longer expiry,WMT,89.29,sell 86.0 - buy 80.0,70,4,2025-03-28,March
9,6,2025-03-08T06:06:42.324030,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,AMZN,189.95,sell 180.0 - buy 170.0,155,1,2025-03-21,March
